In [7]:
import os
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

In [44]:
DIR = os.path.dirname(os.path.abspath(__name__))
DB_PATH = os.path.join(DIR, 'db/')
DB_PATH

'/home/sanchez/VScode_consolidated/task1/db/'

In [45]:
# Connect to the running ChromaDB instance
client = chromadb.PersistentClient(path=DB_PATH)

In [46]:
# Create or get a collection
collection = client.create_collection("documnets_collection")

In [47]:
# Add data to the collection
documents = ["This is a beach", "The forest is green", "Urban areas are crowded"]
metadatas = [{"category": "beach"}, {"category": "forest"}, {"category": "urban"}]
ids = ["1", "2", "3"]

collection.add(documents=documents, metadatas=metadatas, ids=ids)

In [48]:
# Query the collection
results = collection.query(
    query_texts=["What are forests like?"],
    n_results=3  # Number of results to return
)

# Print the results
print(results)

{'ids': [['2', '1', '3']], 'distances': [[0.8161929381010665, 1.6312474679542952, 1.6889317385852014]], 'metadatas': [[{'category': 'forest'}, {'category': 'beach'}, {'category': 'urban'}]], 'embeddings': None, 'documents': [['The forest is green', 'This is a beach', 'Urban areas are crowded']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}
